In [1]:
library(Boruta)
library(dplyr)
library(readr)
library(randomForest)
library(pROC)

Warning message:
"package 'Boruta' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


Warning message:
"package 'pROC' was built under R version 4.4.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




In [2]:
##Read CSV file
whole_data <- read.csv("Soil_MetaFeature_Data.csv")

#Subset DataFrame to include only a specific genus at a time, replace as needed.
subset_data <- whole_data[whole_data$`FUNGI_Taxonomy` == 'Clonostachys', ]

#Prints subset_data to verify what we're grabbing.
subset_data
dim(subset_data)

sampleid                         Filename                             
1   p322_blue_A10_288                p322_blue_A10_288.mzML               
2   p322_blue_A11_213                p322_blue_A11_213.mzML               
3   p322_blue_A12_346                p322_blue_A12_346.mzML               
4   p322_blue_A1_48                  p322_blue_A1_48.mzML                 
5   p322_blue_A2_39                  p322_blue_A2_39.mzML                 
6   p322_blue_A3_240                 p322_blue_A3_240.mzML                
7   p322_blue_A4_316                 p322_blue_A4_316.mzML                
8   p322_blue_A5_241                 p322_blue_A5_241.mzML                
9   p322_blue_A6_328                 p322_blue_A6_328.mzML                
10  p322_blue_A7_180                 p322_blue_A7_180.mzML                
11  p322_blue_A8_347                 p322_blue_A8_347.mzML                
12  p322_blue_A9_52                  p322_blue_A9_52.mzML                 
13  p322_blue_B10_31                 p322_blue_B10_31.mzML                
14  p322_blue_B11_279                p322_blue_B11_279.mzML               
15  p322_blue_B1_127                 p322_blue_B1_127.mzML                
16  p322_blue_B3_16_20240117112014   p322_blue_B3_16_20240117112014.mzML  
17  p322_blue_B4_88                  p322_blue_B4_88.mzML                 
18  p322_blue_B5_86                  p322_blue_B5_86.mzML                 
19  p322_blue_B6_11                  p322_blue_B6_11.mzML                 
20  p322_blue_B7_179                 p322_blue_B7_179.mzML                
21  p322_blue_B8_246                 p322_blue_B8_246.mzML                
22  p322_blue_B9_45                  p322_blue_B9_45.mzML                 
23  p322_blue_C1_343                 p322_blue_C1_343.mzML                
24  p322_blue_C2_202_20240117131927  p322_blue_C2_202_20240117131927.mzML 
25  p322_blue_C3_115                 p322_blue_C3_115.mzML                
26  p322_blue_C4_57                  p322_blue_C4_57.mzML                 
27  p323_green_A10_74                p323_green_A10_74.mzML               
28  p323_green_A11_55                p323_green_A11_55.mzML               
29  p323_green_A12_38                p323_green_A12_38.mzML               
30  p323_green_A1_283_20240117134558 p323_green_A1_283_20240117134558.mzML
⋮   ⋮                                ⋮                                    
292 p332_green_C2_1268               p332_green_C2_1268.mzML              
293 p332_green_C3_1168               p332_green_C3_1168.mzML              
294 p332_green_C4_1267               p332_green_C4_1267.mzML              
295 p333_red_A10_1294                p333_red_A10_1294.mzML               
296 p333_red_A11_1096                p333_red_A11_1096.mzML               
297 p333_red_A12_1207                p333_red_A12_1207.mzML               
298 p333_red_A1_1099                 p333_red_A1_1099.mzML                
299 p333_red_A2_1165                 p333_red_A2_1165.mzML                
300 p333_red_A3_1105                 p333_red_A3_1105.mzML                
301 p333_red_A4_1254                 p333_red_A4_1254.mzML                
302 p333_red_A5_1073                 p333_red_A5_1073.mzML                
303 p333_red_A6_1281                 p333_red_A6_1281.mzML                
304 p333_red_A7_1045                 p333_red_A7_1045.mzML                
305 p333_red_A8_1256                 p333_red_A8_1256.mzML                
306 p333_red_A9_1083                 p333_red_A9_1083.mzML                
307 p333_red_B10_1229                p333_red_B10_1229.mzML               
308 p333_red_B11_1162                p333_red_B11_1162.mzML               
309 p333_red_B12_1157                p333_red_B12_1157.mzML               
310 p333_red_B1_1360                 p333_red_B1_1360.mzML                
311 p333_red_B3_1341                 p333_red_B3_1341.mzML                
312 p333_red_B4_1150                 p333_red_B4_1150.mzML                
313 p333_red_B5_1052         

[1]   321 16375

In [3]:
##################
######332v329#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p323'))

subset_data_plate_1

X_p323 <- subset_data_plate_1[, 12:16374]
y_p323 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p323"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid                         Filename                              Batch
1  p323_green_A10_74                p323_green_A10_74.mzML                NA   
2  p323_green_A11_55                p323_green_A11_55.mzML                NA   
3  p323_green_A12_38                p323_green_A12_38.mzML                NA   
4  p323_green_A1_283_20240117134558 p323_green_A1_283_20240117134558.mzML NA   
5  p323_green_A2_272                p323_green_A2_272.mzML                NA   
6  p323_green_A3_255                p323_green_A3_255.mzML                NA   
7  p323_green_A4_148                p323_green_A4_148.mzML                NA   
8  p323_green_A5_18                 p323_green_A5_18.mzML                 NA   
9  p323_green_A6_278                p323_green_A6_278.mzML                NA   
10 p323_green_A7_98                 p323_green_A7_98.mzML                 NA   
11 p323_green_A8_300                p323_green_A8_300.mzML                NA   
12 p323_green_A9_117                p323_green_A9_117.mzML                NA   
13 p323_green_B10_284               p323_green_B10_284.mzML               NA   
14 p323_green_B11_309               p323_green_B11_309.mzML               NA   
15 p323_green_B12_321               p323_green_B12_321.mzML               NA   
16 p323_green_B1_76                 p323_green_B1_76.mzML                 NA   
17 p323_green_B3_245                p323_green_B3_245.mzML                NA   
18 p323_green_B4_292                p323_green_B4_292.mzML                NA   
19 p323_green_B5_166                p323_green_B5_166.mzML                NA   
20 p323_green_B6_247                p323_green_B6_247.mzML                NA   
21 p323_green_B7_322                p323_green_B7_322.mzML                NA   
22 p323_green_B8_307                p323_green_B8_307.mzML                NA   
23 p323_green_B9_95                 p323_green_B9_95.mzML                 NA   
24 p323_green_C1_53                 p323_green_C1_53.mzML                 NA   
25 p323_green_C2_244                p323_green_C2_244.mzML                NA   
26 p323_green_C3_144                p323_green_C3_144.mzML                NA   
27 p323_green_C4_243                p323_green_C4_243.mzML                NA   
28 p332_green_A10_1098              p332_green_A10_1098.mzML              NA   
29 p332_green_A11_1079              p332_green_A11_1079.mzML              NA   
30 p332_green_A12_1062              p332_green_A12_1062.mzML              NA   
31 p332_green_A1_1308               p332_green_A1_1308.mzML               NA   
32 p332_green_A2_1296               p332_green_A2_1296.mzML               NA   
33 p332_green_A3_1279               p332_green_A3_1279.mzML               NA   
34 p332_green_A4_1172               p332_green_A4_1172.mzML               NA   
35 p332_green_A5_1042               p332_green_A5_1042.mzML               NA   
36 p332_green_A6_1303               p332_green_A6_1303.mzML               NA   
37 p332_green_A7_1123               p332_green_A7_1123.mzML               NA   
38 p332_green_A8_1325               p332_green_A8_1325.mzML               NA   
39 p332_green_A9_1141               p332_green_A9_1141.mzML               NA   
40 p332_green_B10_1309              p332_green_B10_1309.mzML              NA   
41 p332_green_B11_1333              p332_green_B11_1333.mzML              NA   
42 p332_green_B1_1100               p332_green_B1_1100.mzML               NA   
43 p332_green_B3_1269               p332_green_B3_1269.mzML               NA   
44 p332_green_B4_1317               p332_green_B4_1317.mzML               NA   
45 p332_green_B5_1190               p332_green_B5_1190.mzML               NA   
46 p332_green_B6_1271               p332_green_B6_1271.mzML               NA   
47 p332_green_B7_1346               p332_green_B7_1346.mzML               NA   
48 p332_green_B8_1331               p332_green_B8_1331.mzML               NA   
49 p332_green_B9_1120               p332_green_B9_1120.mzML               NA   
50 

In [4]:
#Run Boruta for p322vTarget plate group
boruta_p323 <- Boruta(X_p323, y_p323, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p323, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p323)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_7_LPS_Clono_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +43 secs: 

 confirmed 7 attributes: X174.1129_2.33_3230, X236.1496_0.25_2337, X282.1199_0.47_3145, X285.1314_3.25_4766, X307.1267_3.03_9422 and 2 more;

 rejected 16044 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X

In [5]:
##################
######329v326#####
##################

#Further subsets to only include groups of interest.
subset_data_plate_2 <- subset_data %>%
  filter(plate %in% c('p329', 'p326'))

subset_data_plate_2

X_p326 <- subset_data_plate_2[, 12:16374]
y_p326 <- as.factor(subset(subset_data_plate_2, plate %in% c("p329", "p326"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p326_green_A10_416  p326_green_A10_416.mzML  NA    sample 416      p326 
2  p326_green_A11_396  p326_green_A11_396.mzML  NA    sample 396      p326 
3  p326_green_A12_379  p326_green_A12_379.mzML  NA    sample 379      p326 
4  p326_green_A1_625   p326_green_A1_625.mzML   NA    sample 625      p326 
5  p326_green_A2_614   p326_green_A2_614.mzML   NA    sample 614      p326 
6  p326_green_A3_596   p326_green_A3_596.mzML   NA    sample 596      p326 
7  p326_green_A4_489   p326_green_A4_489.mzML   NA    sample 489      p326 
8  p326_green_A5_359   p326_green_A5_359.mzML   NA    sample 359      p326 
9  p326_green_A6_620   p326_green_A6_620.mzML   NA    sample 620      p326 
10 p326_green_A7_440   p326_green_A7_440.mzML   NA    sample 440      p326 
11 p326_green_A8_642   p326_green_A8_642.mzML   NA    sample 642      p326 
12 p326_green_A9_458   p326_green_A9_458.mzML   NA    sample 458      p326 
13 p326_green_B10_626  p326_green_B10_626.mzML  NA    sample 626      p326 
14 p326_green_B11_650  p326_green_B11_650.mzML  NA    sample 650      p326 
15 p326_green_B1_418   p326_green_B1_418.mzML   NA    sample 418      p326 
16 p326_green_B3_586   p326_green_B3_586.mzML   NA    sample 586      p326 
17 p326_green_B4_634   p326_green_B4_634.mzML   NA    sample 634      p326 
18 p326_green_B5_508   p326_green_B5_508.mzML   NA    sample 508      p326 
19 p326_green_B6_588   p326_green_B6_588.mzML   NA    sample 588      p326 
20 p326_green_B7_663   p326_green_B7_663.mzML   NA    sample 663      p326 
21 p326_green_B8_648   p326_green_B8_648.mzML   NA    sample 648      p326 
22 p326_green_B9_437   p326_green_B9_437.mzML   NA    sample 437      p326 
23 p326_green_C1_394   p326_green_C1_394.mzML   NA    sample 394      p326 
24 p326_green_C2_585   p326_green_C2_585.mzML   NA    sample 585      p326 
25 p326_green_C3_485   p326_green_C3_485.mzML   NA    sample 485      p326 
26 p326_green_C4_584   p326_green_C4_584.mzML   NA    sample 584      p326 
27 p329_green_A10_757  p329_green_A10_757.mzML  NA    sample 757      p329 
28 p329_green_A11_738  p329_green_A11_738.mzML  NA    sample 738      p329 
29 p329_green_A12_721  p329_green_A12_721.mzML  NA    sample 721      p329 
30 p329_green_A1_966   p329_green_A1_966.mzML   NA    sample 966      p329 
31 p329_green_A2_955   p329_green_A2_955.mzML   NA    sample 955      p329 
32 p329_green_A3_938   p329_green_A3_938.mzML   NA    sample 938      p329 
33 p329_green_A4_831   p329_green_A4_831.mzML   NA    sample 831      p329 
34 p329_green_A5_700   p329_green_A5_700.mzML   NA    sample 700      p329 
35 p329_green_A6_961   p329_green_A6_961.mzML   NA    sample 961      p329 
36 p329_green_A7_781   p329_green_A7_781.mzML   NA    sample 781      p329 
37 p329_green_A8_983   p329_green_A8_983.mzML   NA    sample 983      p329 
38 p329_green_A9_799   p329_green_A9_799.mzML   NA    sample 799      p329 
39 p329_green_B10_967  p329_green_B10_967.mzML  NA    sample 967      p329 
40 p329_green_B11_991  p329_green_B11_991.mzML  NA    sample 991      p329 
41 p329_green_B12_1004 p329_green_B12_1004.mzML NA    sample 1004     p329 
42 p329_green_B1_759   p329_green_B1_759.mzML   NA    sample 759      p329 
43 p329_green_B3_928   p329_green_B3_928.mzML   NA    sample 928      p329 
44 p329_green_B4_975   p329_green_B4_975.mzML   NA    sample 975      p329 
45 p329_green_B5_849   p329_green_B5_849.mzML   NA    sample 849      p329 
46 p329_green_B6_930   p329_green_B6_930.mzML   NA    sample 930      p329 
47 p329_green_B7_1005  p329_green_B7_1005.mzML  NA    sample 1005     p329 
48 p329_green_B8_989   p329_green_B8_989.mzML   NA    sample 989      p329 
49 p329_green_B9_778   p329_green_B9_778.mzML   NA    sample 778      p329 
50 p329_green_C1_736   p329_green_C1_736.mzML   NA    sample 736      p329 
51 p329_green_C2_927   p329_green_C2_927.mzML   NA    sample 927      p329 
52 p329_green_C3_827   p329_green_C3_827.mzML   NA 

In [6]:
#Run Boruta for p3XXvTarget plate group
boruta_p326 <- Boruta(X_p326, y_p326, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p326, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p326)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_LPS_Clono_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +44 secs: 

 rejected 16223 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16218 more;

 still have 140 attributes left.


 22. run of importance source...

 23. run of importance source...

 

In [7]:
##################
######332v329#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p329'))

subset_data_plate_1

X_p329 <- subset_data_plate_1[, 12:16374]
y_p329 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p329"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p329_green_A10_757  p329_green_A10_757.mzML  NA    sample 757      p329 
2  p329_green_A11_738  p329_green_A11_738.mzML  NA    sample 738      p329 
3  p329_green_A12_721  p329_green_A12_721.mzML  NA    sample 721      p329 
4  p329_green_A1_966   p329_green_A1_966.mzML   NA    sample 966      p329 
5  p329_green_A2_955   p329_green_A2_955.mzML   NA    sample 955      p329 
6  p329_green_A3_938   p329_green_A3_938.mzML   NA    sample 938      p329 
7  p329_green_A4_831   p329_green_A4_831.mzML   NA    sample 831      p329 
8  p329_green_A5_700   p329_green_A5_700.mzML   NA    sample 700      p329 
9  p329_green_A6_961   p329_green_A6_961.mzML   NA    sample 961      p329 
10 p329_green_A7_781   p329_green_A7_781.mzML   NA    sample 781      p329 
11 p329_green_A8_983   p329_green_A8_983.mzML   NA    sample 983      p329 
12 p329_green_A9_799   p329_green_A9_799.mzML   NA    sample 799      p329 
13 p329_green_B10_967  p329_green_B10_967.mzML  NA    sample 967      p329 
14 p329_green_B11_991  p329_green_B11_991.mzML  NA    sample 991      p329 
15 p329_green_B12_1004 p329_green_B12_1004.mzML NA    sample 1004     p329 
16 p329_green_B1_759   p329_green_B1_759.mzML   NA    sample 759      p329 
17 p329_green_B3_928   p329_green_B3_928.mzML   NA    sample 928      p329 
18 p329_green_B4_975   p329_green_B4_975.mzML   NA    sample 975      p329 
19 p329_green_B5_849   p329_green_B5_849.mzML   NA    sample 849      p329 
20 p329_green_B6_930   p329_green_B6_930.mzML   NA    sample 930      p329 
21 p329_green_B7_1005  p329_green_B7_1005.mzML  NA    sample 1005     p329 
22 p329_green_B8_989   p329_green_B8_989.mzML   NA    sample 989      p329 
23 p329_green_B9_778   p329_green_B9_778.mzML   NA    sample 778      p329 
24 p329_green_C1_736   p329_green_C1_736.mzML   NA    sample 736      p329 
25 p329_green_C2_927   p329_green_C2_927.mzML   NA    sample 927      p329 
26 p329_green_C3_827   p329_green_C3_827.mzML   NA    sample 827      p329 
27 p329_green_C4_926   p329_green_C4_926.mzML   NA    sample 926      p329 
28 p332_green_A10_1098 p332_green_A10_1098.mzML NA    sample 1098     p332 
29 p332_green_A11_1079 p332_green_A11_1079.mzML NA    sample 1079     p332 
30 p332_green_A12_1062 p332_green_A12_1062.mzML NA    sample 1062     p332 
31 p332_green_A1_1308  p332_green_A1_1308.mzML  NA    sample 1308     p332 
32 p332_green_A2_1296  p332_green_A2_1296.mzML  NA    sample 1296     p332 
33 p332_green_A3_1279  p332_green_A3_1279.mzML  NA    sample 1279     p332 
34 p332_green_A4_1172  p332_green_A4_1172.mzML  NA    sample 1172     p332 
35 p332_green_A5_1042  p332_green_A5_1042.mzML  NA    sample 1042     p332 
36 p332_green_A6_1303  p332_green_A6_1303.mzML  NA    sample 1303     p332 
37 p332_green_A7_1123  p332_green_A7_1123.mzML  NA    sample 1123     p332 
38 p332_green_A8_1325  p332_green_A8_1325.mzML  NA    sample 1325     p332 
39 p332_green_A9_1141  p332_green_A9_1141.mzML  NA    sample 1141     p332 
40 p332_green_B10_1309 p332_green_B10_1309.mzML NA    sample 1309     p332 
41 p332_green_B11_1333 p332_green_B11_1333.mzML NA    sample 1333     p332 
42 p332_green_B1_1100  p332_green_B1_1100.mzML  NA    sample 1100     p332 
43 p332_green_B3_1269  p332_green_B3_1269.mzML  NA    sample 1269     p332 
44 p332_green_B4_1317  p332_green_B4_1317.mzML  NA    sample 1317     p332 
45 p332_green_B5_1190  p332_green_B5_1190.mzML  NA    sample 1190     p332 
46 p332_green_B6_1271  p332_green_B6_1271.mzML  NA    sample 1271     p332 
47 p332_green_B7_1346  p332_green_B7_1346.mzML  NA    sample 1346     p332 
48 p332_green_B8_1331  p332_green_B8_1331.mzML  NA    sample 1331     p332 
49 p332_green_B9_1120  p332_green_B9_1120.mzML  NA    sample 1120     p332 
50 p332_green_C1_1077  p332_green_C1_1077.mzML  NA    sample 1077     p332 
51 p332_green_C2_1268  p332_green_C2_1268.mzML  NA    sample 1268     p332 
52 p332_green_C3_1168  p332_green_C3_1168.mzML  NA 

In [8]:
#Run Boruta for p322vTarget plate group
boruta_p329 <- Boruta(X_p329, y_p329, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p329, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p329)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_No_LPS_Clono.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +43 secs: 

 confirmed 3 attributes: X202.0715_2.38_12527, X381.2623_4.46_17557, X781.4369_5.61_8079;

 rejected 15886 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 15881 more;

 still have 4